Do not use this! The working evaluation for the fine-tuned BART model is implemented in scripts/zero_shot_all_labels_fine_tune.py

In [3]:
model_dir = "/root/models/bart_frozen_base_3_12_23"

In [4]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    
print_gpu_utilization()

GPU memory occupied: 438 MB.


In [1]:
from transformers import pipeline
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
import datasets
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import os
os.environ['DISABLE_MLFLOW_INTEGRATION'] = 'TRUE'

print_gpu_utilization()

NameError: name 'print_gpu_utilization' is not defined

In [7]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli", local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(model_dir, local_files_only=True)
print_gpu_utilization()
model.config.label2id

GPU memory occupied: 438 MB.


{'contradiction': 0, 'entailment': 2, 'neutral': 1}

In [28]:
# train_tokenized = datasets.load_from_disk("/root/data/bart_train_labels")
# val_tokenized = datasets.load_from_disk("/root/data/bart_val_labels")

# from datasets import concatenate_datasets
# train_concat = concatenate_datasets([train_tokenized['train'], train_tokenized['val']])
# val_concat = concatenate_datasets([val_tokenized['train'], val_tokenized['val']])


# small_train_dataset = train_concat.shuffle(seed=42)
# small_val_dataset = val_concat.shuffle(seed=42).select(range(10))

In [2]:
train_tokenized = datasets.load_from_disk("/root/data/bart_train_labels")
val_tokenized = datasets.load_from_disk("/root/data/bart_val_labels")
test_tokenized = datasets.load_from_disk("/root/data/bart_test_labels")
train_tokenized

DatasetDict({
    train: Dataset({
        features: ['id', 'labels', 'original_label', 'input_ids', 'attention_mask'],
        num_rows: 511520
    })
    val: Dataset({
        features: ['id', 'labels', 'original_label', 'input_ids', 'attention_mask'],
        num_rows: 146150
    })
    test: Dataset({
        features: ['id', 'labels', 'original_label', 'input_ids', 'attention_mask'],
        num_rows: 73075
    })
})

In [12]:
small_val_dataset = val_tokenized["val"].shuffle(seed=42).select(range(10))

Loading cached shuffled indices for dataset at /root/data/bart_val_labels/val/cache-7e5c3367322bc272.arrow


In [19]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits_tuple, labels = eval_pred
    logits, _ = logits_tuple
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir=f"{model_dir}/output",
    evaluation_strategy="epoch",
    num_train_epochs=1,
    logging_steps=10,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=4, # TODO: adjust
    gradient_accumulation_steps=10, # effective batch size is per_device_train_batch_size * gradient_accumulation_steps
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=small_val_dataset,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
predictions = trainer.predict(small_val_dataset)

The following columns in the test set don't have a corresponding argument in `BartForSequenceClassification.forward` and have been ignored: id, original_label. If id, original_label are not expected by `BartForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10
  Batch size = 4


In [ ]:
import pickle
# for dataset, name in zip([train_tokenized, val_tokenized, test_tokenized], ["train", "val", "test"]):
for dataset, name in zip([small_val_dataset], ["zzz"]):
    print(f"begin {name}")
    predictions = trainer.predict(dataset)
    
    # open a file, where you ant to store the data
    file = open(f'/root/data/bart_fine_tune_pred/{name}.csv', 'wb')

    # dump information to that file
    pickle.dump(data, file)

    # close the file
    file.close()
    print(f"end {name}")

In [25]:
import pickle

# open a file, where you stored the pickled data
file = open(f'/root/data/bart_fine_tune_pred/zzz.csv', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()
data

PredictionOutput(predictions=(array([[-3.0188463,  3.8209403, -1.3312486],
       [-4.6022134,  6.457174 , -2.3633356],
       [-1.8877857,  2.5640533, -1.138127 ],
       [-2.2623603,  5.341508 , -3.3918443],
       [-2.62325  ,  3.0805368, -1.234132 ],
       [-1.6223269,  4.2531533, -3.119474 ],
       [-3.0899057,  4.8664193, -1.9516855],
       [-3.342742 ,  4.076881 , -1.359197 ],
       [-2.9088228,  4.1623178, -1.8885658],
       [-4.0659566,  5.1598377, -1.5097034]], dtype=float32), array([[[ 5.33955405e-03,  3.81233841e-02,  2.86328364e-02, ...,
          1.56229492e-02,  6.21573487e-03, -1.65023957e-03],
        [-2.43430808e-01,  5.03124446e-02, -2.65179813e-01, ...,
         -5.84626421e-02,  1.53724223e-01, -1.01395264e-01],
        [-6.28883252e-03,  2.62500998e-03,  3.67596932e-03, ...,
          6.30748691e-03, -3.26891057e-03, -1.12911453e-04],
        ...,
        [ 5.33954380e-03,  3.81234027e-02,  2.86328327e-02, ...,
          1.56229557e-02,  6.21572277e-03, -1.6